# 07. LIBERO Simulation Setup

**Goal**: Set up the LIBERO simulation environment for evaluating OpenVLA.

## What We'll Learn
1. LIBERO benchmark overview
2. Environment setup and configuration
3. Task suites and scenarios
4. Running rollouts in simulation
5. Recording videos for analysis

---
## 1. LIBERO Benchmark Overview

**LIBERO** (LIfelong robot BEnchmark for RObotics) is a simulation benchmark for evaluating robot manipulation policies.

In [ ]:
libero_overview = """
┌────────────────────────────────────────────────────────────────────┐
│                      LIBERO Benchmark                               │
├────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  4 Task Suites, 90 Total Tasks                                      │
│                                                                     │
│  ┌──────────────────┬───────────────────────────────────────────┐  │
│  │ LIBERO-Spatial   │ 10 tasks with spatial variations          │  │
│  │                  │ Same objects, different positions          │  │
│  ├──────────────────┼───────────────────────────────────────────┤  │
│  │ LIBERO-Object    │ 10 tasks with object variations           │  │
│  │                  │ Same positions, different objects          │  │
│  ├──────────────────┼───────────────────────────────────────────┤  │
│  │ LIBERO-Goal      │ 10 tasks with goal variations             │  │
│  │                  │ Same setup, different target goals         │  │
│  ├──────────────────┼───────────────────────────────────────────┤  │
│  │ LIBERO-90        │ Full 90-task benchmark                    │  │
│  │                  │ Comprehensive evaluation                   │  │
│  └──────────────────┴───────────────────────────────────────────┘  │
│                                                                     │
│  Robot: Franka Emika Panda (7-DoF arm + gripper)                   │
│  Physics: MuJoCo via robosuite                                      │
│  Observation: 256×256 RGB image (agentview)                        │
│  Action: 7-DoF (6D pose delta + gripper)                           │
│                                                                     │
└────────────────────────────────────────────────────────────────────┘
"""
print(libero_overview)

---
## 2. Environment Setup

In [ ]:
# ============================================================
# CRITICAL: Set these BEFORE importing any packages!
# ============================================================
import os

# For NERSC Perlmutter, use your $PSCRATCH directory
PSCRATCH = "/pscratch/sd/d/dpark1"  # CHANGE THIS TO YOUR PATH
CACHE_DIR = f"{PSCRATCH}/.cache"

# Set all cache directories to $PSCRATCH/.cache
os.environ['XDG_CACHE_HOME'] = CACHE_DIR
os.environ['HF_HOME'] = f"{CACHE_DIR}/huggingface"
os.environ['TFDS_DATA_DIR'] = f"{CACHE_DIR}/tensorflow_datasets"
os.environ['TORCH_HOME'] = f"{CACHE_DIR}/torch"

# ============================================================
# MuJoCo/OpenGL rendering setup - MUST be set before imports!
# ============================================================
# Option 1: OSMesa (CPU rendering, headless) - requires libosmesa
# Option 2: EGL (GPU rendering, headless) - requires GPU + EGL libs
# Option 3: GLFW (display rendering) - requires display

# For NERSC Perlmutter GPU nodes, try EGL first (faster)
# For CPU-only or if EGL fails, use OSMesa
RENDER_MODE = "egl"  # Change to "osmesa" if EGL doesn't work

os.environ['MUJOCO_GL'] = RENDER_MODE
os.environ['PYOPENGL_PLATFORM'] = RENDER_MODE  # Must match MUJOCO_GL!

# Create directories
for path in [CACHE_DIR, os.environ['HF_HOME'], os.environ['TFDS_DATA_DIR'], os.environ['TORCH_HOME']]:
    os.makedirs(path, exist_ok=True)

print(f"✅ All caches → {CACHE_DIR}")
print(f"✅ MUJOCO_GL = {os.environ.get('MUJOCO_GL')}")
print(f"✅ PYOPENGL_PLATFORM = {os.environ.get('PYOPENGL_PLATFORM')}")
print("")
print("If you see OpenGL errors, try changing RENDER_MODE to 'osmesa' or 'egl'")
print("For OSMesa: module load osmesa  # or apt-get install libosmesa6-dev")
print("For EGL: requires GPU node with EGL support")

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time

# Import LIBERO - this may take a moment
print("Importing LIBERO (this loads MuJoCo and robosuite)...")
try:
    from libero.libero import benchmark
    from libero.libero.envs import OffScreenRenderEnv
    print("✅ LIBERO imported successfully!")
except ImportError as e:
    print(f"❌ LIBERO import error: {e}")
    print("\nInstall with: pip install libero")
except AttributeError as e:
    if "glGetError" in str(e) or "GL" in str(e):
        print(f"❌ OpenGL rendering error: {e}")
        print("\nTroubleshooting:")
        print("1. Try changing RENDER_MODE in the cell above:")
        print("   - 'egl' for GPU nodes (faster)")
        print("   - 'osmesa' for CPU rendering")
        print("")
        print("2. For NERSC Perlmutter, load required modules:")
        print("   module load cudatoolkit  # for EGL")
        print("   # or")
        print("   module load osmesa  # for OSMesa")
        print("")
        print("3. Restart the kernel after changing RENDER_MODE")
    else:
        raise

In [ ]:
# Import robosuite
try:
    import robosuite
    print(f"robosuite version: {robosuite.__version__}")
except ImportError as e:
    print(f"robosuite import error: {e}")
    print("Install with: pip install robosuite==1.4.1")

In [ ]:
# Import MuJoCo
try:
    import mujoco
    print(f"MuJoCo version: {mujoco.__version__}")
except ImportError as e:
    print(f"MuJoCo import error: {e}")
    print("Install with: pip install mujoco")

---
## 3. Load LIBERO Task Suites

In [ ]:
# Explore LIBERO benchmark API thoroughly
print("="*60)
print("LIBERO Benchmark API Exploration")
print("="*60)

# 1. List ALL available items in benchmark module
print("\n1. Everything in benchmark module:")
all_items = [m for m in dir(benchmark) if not m.startswith('_')]
for item in all_items:
    obj = getattr(benchmark, item)
    print(f"   {item}: {type(obj).__name__}")

# 2. Check for common class names
print("\n2. Looking for Benchmark classes:")
for name in ['Benchmark', 'BENCHMARK', 'BenchmarkDict', 'Task', 'TASK']:
    if hasattr(benchmark, name):
        print(f"   Found: {name}")
        obj = getattr(benchmark, name)
        print(f"      Type: {type(obj)}")
        if isinstance(obj, dict):
            print(f"      Keys: {list(obj.keys())}")

# 3. Try get_benchmark_dict and explore its return value
print("\n3. Exploring get_benchmark_dict('libero_spatial'):")
result = benchmark.get_benchmark_dict('libero_spatial')
print(f"   Type: {type(result)}")
print(f"   Value: {result}")

# If it's a class/object, explore its methods
if hasattr(result, '__dict__'):
    print(f"   __dict__: {result.__dict__}")

# 4. Try different API patterns
print("\n4. Trying different API patterns:")

# Pattern A: get_benchmark returns a Benchmark object
if hasattr(benchmark, 'get_benchmark'):
    print("   Found: get_benchmark()")
    try:
        bench = benchmark.get_benchmark('libero_spatial')
        print(f"      Returns: {type(bench)}")
        print(f"      Methods: {[m for m in dir(bench) if not m.startswith('_')]}")
    except Exception as e:
        print(f"      Error: {e}")

# Pattern B: Benchmark class that you instantiate
for cls_name in ['Benchmark', 'LIBERO', 'LiberoSuite', 'TaskSuite']:
    if hasattr(benchmark, cls_name):
        cls = getattr(benchmark, cls_name)
        print(f"   Found class: {cls_name}")
        try:
            instance = cls('libero_spatial')
            print(f"      Instance methods: {[m for m in dir(instance) if not m.startswith('_')]}")
        except Exception as e:
            print(f"      Instantiation error: {e}")

# Pattern C: Direct task access functions
for func_name in ['get_task', 'get_tasks', 'load_task', 'get_task_class', 'get_task_names', 'get_task_bddls']:
    if hasattr(benchmark, func_name):
        print(f"   Found: {func_name}()")

# 5. Check if benchmark itself is callable or iterable
print("\n5. Checking benchmark module properties:")
print(f"   Is callable: {callable(benchmark)}")
print(f"   Has __iter__: {hasattr(benchmark, '__iter__')}")
print(f"   Module file: {getattr(benchmark, '__file__', 'N/A')}")

In [ ]:
# Correct API: benchmark.get_benchmark() returns a CLASS that must be INSTANTIATED
# BenchmarkClass = benchmark.get_benchmark('libero_spatial')
# bench_instance = BenchmarkClass()  # <-- Need to instantiate!
# task = bench_instance.get_task(task_id)

def get_benchmark_instance(suite_name):
    """Get an instantiated benchmark object for a suite."""
    BenchmarkClass = benchmark.get_benchmark(suite_name)
    return BenchmarkClass()  # Instantiate the class

def get_task_names(suite_name):
    """Get task names from a LIBERO suite."""
    bench = get_benchmark_instance(suite_name)
    return bench.get_task_names()

def get_task(suite_name, task_id):
    """Get a specific task from a suite."""
    bench = get_benchmark_instance(suite_name)
    return bench.get_task(task_id)

def get_num_tasks(suite_name):
    """Get number of tasks in a suite."""
    bench = get_benchmark_instance(suite_name)
    return bench.get_num_tasks()

# Test the API
print("Testing LIBERO API:")
print("="*60)

task_suites = ["libero_spatial", "libero_object", "libero_goal", "libero_90"]

for suite_name in task_suites:
    try:
        n_tasks = get_num_tasks(suite_name)
        names = get_task_names(suite_name)
        print(f"\n{suite_name}: {n_tasks} tasks")
        for name in names[:3]:
            print(f"   - {name}")
        if len(names) > 3:
            print(f"   ... and {len(names)-3} more")
    except Exception as e:
        print(f"\n{suite_name}: Error - {e}")

---
## 4. Create a LIBERO Environment

In [ ]:
# Select a task suite
SUITE_NAME = "libero_spatial"

task_names = get_task_names(SUITE_NAME)
n_tasks = get_num_tasks(SUITE_NAME)

print(f"Selected suite: {SUITE_NAME}")
print(f"Number of tasks: {n_tasks}")
print(f"\nAll tasks in {SUITE_NAME}:")
for i, name in enumerate(task_names):
    print(f"  {i}: {name}")

In [ ]:
# Select a specific task
TASK_ID = 0

# Get task using our helper function
task = get_task(SUITE_NAME, TASK_ID)

print("Task Configuration:")
print("="*60)
print(f"Task ID: {TASK_ID}")
print(f"Name: {task.name}")
print(f"Language instruction: {task.language}")
print(f"BDDL file: {task.bddl_file}")

In [ ]:
# LIBERO requires configuration for asset paths
# The config file must be in libero/libero/.libero_config (same dir as __init__.py)

import yaml
import libero
import libero.libero

# The config file location is where libero.libero module is
libero_libero_path = os.path.dirname(libero.libero.__file__)
libero_config_path = os.path.join(libero_libero_path, ".libero_config")

print(f"LIBERO libero submodule path: {libero_libero_path}")
print(f"LIBERO config file: {libero_config_path}")

# Find the bddl_files directory
bddl_path = os.path.join(libero_libero_path, "bddl_files")

print(f"\nAsset paths:")
print(f"  bddl_files: {bddl_path} (exists: {os.path.exists(bddl_path)})")

# Create the config file
config = {
    'bddl_files': bddl_path,
}

print(f"\nCreating config file with: {config}")

with open(libero_config_path, 'w') as f:
    yaml.dump(config, f)

print(f"✅ Config file created at: {libero_config_path}")

# Verify content
with open(libero_config_path, 'r') as f:
    print(f"Config content: {f.read()}")

# Verify it works
from libero.libero import get_libero_path
try:
    bddl_check = get_libero_path("bddl_files")
    print(f"✅ get_libero_path('bddl_files') = {bddl_check}")
except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
# Now create the environment
def create_libero_env(task, suite_name, image_size=256):
    """
    Create a LIBERO environment for a given task.
    """
    # Get the full path to the BDDL file
    bench = get_benchmark_instance(suite_name)
    bddl_file_path = bench.get_task_bddl_file_path(task)
    
    print(f"BDDL file path: {bddl_file_path}")
    
    env_args = {
        "bddl_file_name": bddl_file_path,
        "camera_heights": image_size,
        "camera_widths": image_size,
    }
    
    env = OffScreenRenderEnv(**env_args)
    env.seed(0)
    
    return env

print("Creating environment...")
env = create_libero_env(task, SUITE_NAME)
print("✅ Environment created!")

In [ ]:
# Explore environment properties
print("Environment Properties:")
print("="*60)
print(f"Action space: {env.action_spec}")
print(f"Action dimension: {env.action_dim}")
print(f"\nObservation keys:")

# Reset to get observation structure
obs = env.reset()
for key, value in obs.items():
    if isinstance(value, np.ndarray):
        print(f"  {key}: shape={value.shape}, dtype={value.dtype}")
    else:
        print(f"  {key}: {type(value)}")

---
## 5. Visualize the Environment

In [ ]:
# Render initial observation
def get_observation_image(obs, key='agentview_image'):
    """
    Extract RGB image from observation dict.
    
    LIBERO convention: Images need to be rotated 180 degrees.
    """
    image = obs[key]
    
    # LIBERO images are upside down - rotate 180 degrees
    image = np.rot90(image, k=2)
    
    return image

# Get and display the initial observation
obs = env.reset()
image = get_observation_image(obs)

plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.title(f"Initial Observation\nTask: {task.language}")
plt.axis('off')
plt.show()

print(f"Image shape: {image.shape}")

In [ ]:
# Execute some random actions to see the environment dynamics
def run_random_rollout(env, n_steps=10):
    """
    Run a random rollout and collect images.
    """
    obs = env.reset()
    images = [get_observation_image(obs)]
    
    for _ in range(n_steps):
        # Random action
        action = np.random.uniform(-0.1, 0.1, size=env.action_dim)
        action[-1] = 0  # Keep gripper open
        
        obs, reward, done, info = env.step(action)
        images.append(get_observation_image(obs))
        
        if done:
            break
    
    return images

# Run random rollout
print("Running random rollout (10 steps)...")
rollout_images = run_random_rollout(env, n_steps=10)

# Display frames
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i, (ax, img) in enumerate(zip(axes.flat, rollout_images[:10])):
    ax.imshow(img)
    ax.set_title(f"Step {i}")
    ax.axis('off')
plt.suptitle("Random Rollout")
plt.tight_layout()
plt.show()

---
## 6. Action Space Details

In [ ]:
action_space_info = """
LIBERO Action Space (7-DoF)
============================

Action dimensions:
  [0] dx:      End-effector X displacement
  [1] dy:      End-effector Y displacement  
  [2] dz:      End-effector Z displacement
  [3] d_roll:  Rotation around X axis
  [4] d_pitch: Rotation around Y axis
  [5] d_yaw:   Rotation around Z axis
  [6] gripper: Gripper action (-1: open, 1: close)

Action bounds (typical):
  Position: [-0.05, 0.05] meters per step
  Rotation: [-0.17, 0.17] radians per step  
  Gripper:  [-1.0, 1.0]

Note: OpenVLA outputs normalized actions in [-1, 1]
that need to be scaled to these bounds.
"""
print(action_space_info)

In [ ]:
# Examine action bounds
print("Environment Action Specification:")
print("="*60)

action_spec = env.action_spec
print(f"Action dimension: {env.action_dim}")
print(f"Action spec shape: {action_spec[0].shape}")

# Typical LIBERO action bounds
libero_action_bounds = {
    'position': (-0.05, 0.05),    # meters
    'rotation': (-0.17, 0.17),    # radians (~10 degrees)
    'gripper': (-1.0, 1.0),
}

print(f"\nTypical action bounds:")
for dim, bounds in libero_action_bounds.items():
    print(f"  {dim}: [{bounds[0]}, {bounds[1]}]")

---
## 7. Image Preprocessing for OpenVLA

In [ ]:
# LIBERO image preprocessing to match OpenVLA training
import io

def preprocess_libero_image_for_openvla(obs, key='agentview_image', target_size=224):
    """
    Preprocess LIBERO observation image for OpenVLA inference.
    
    This matches the preprocessing used during OpenVLA training:
    1. Rotate 180 degrees (LIBERO convention)
    2. Encode as JPEG and decode (matches training augmentation)
    3. Resize with Lanczos interpolation
    """
    # Get image from observation
    image = obs[key]
    
    # Rotate 180 degrees (LIBERO images are upside down)
    image = np.rot90(image, k=2)
    
    # Convert to PIL Image
    pil_image = Image.fromarray(image.astype(np.uint8))
    
    # JPEG encode/decode (matches training preprocessing)
    buffer = io.BytesIO()
    pil_image.save(buffer, format='JPEG', quality=95)
    buffer.seek(0)
    pil_image = Image.open(buffer)
    
    # Resize to target size
    pil_image = pil_image.resize((target_size, target_size), Image.LANCZOS)
    
    return pil_image

# Test preprocessing
obs = env.reset()
processed_image = preprocess_libero_image_for_openvla(obs)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(get_observation_image(obs))
axes[0].set_title(f"Raw (256×256)")
axes[0].axis('off')

axes[1].imshow(processed_image)
axes[1].set_title(f"Preprocessed for OpenVLA (224×224)")
axes[1].axis('off')

plt.tight_layout()
plt.show()

---
## 8. Running a Policy Rollout

In [ ]:
def run_policy_rollout(
    env,
    policy_fn,
    instruction,
    max_steps=400,
    record_video=True
):
    """
    Run a policy rollout in the environment.
    
    Args:
        env: LIBERO environment
        policy_fn: Function(image, instruction) -> action
        instruction: Task instruction string
        max_steps: Maximum steps per episode
        record_video: Whether to record frames
    
    Returns:
        success: Whether task was completed
        frames: List of observation images (if record_video)
        actions: List of executed actions
    """
    obs = env.reset()
    
    frames = []
    actions = []
    success = False
    
    for step in range(max_steps):
        # Get preprocessed image
        image = preprocess_libero_image_for_openvla(obs)
        
        if record_video:
            frames.append(get_observation_image(obs))
        
        # Get action from policy
        action = policy_fn(image, instruction)
        actions.append(action)
        
        # Execute action
        obs, reward, done, info = env.step(action)
        
        if done:
            success = info.get('success', reward > 0)
            break
    
    if record_video:
        frames.append(get_observation_image(obs))
    
    return success, frames, actions

# Example: Run with random policy
def random_policy(image, instruction):
    """Random policy for testing."""
    action = np.random.uniform(-0.05, 0.05, size=7)
    return action

print("Running random policy rollout...")
success, frames, actions = run_policy_rollout(
    env, 
    random_policy, 
    task.language,
    max_steps=50,
)

print(f"\nRollout completed:")
print(f"  Success: {success}")
print(f"  Steps: {len(actions)}")
print(f"  Frames recorded: {len(frames)}")

---
## 9. Video Recording Utilities

In [ ]:
def save_video(frames, output_path, fps=20):
    """
    Save frames as MP4 video.
    
    Args:
        frames: List of numpy arrays (H, W, 3)
        output_path: Path to save video
        fps: Frames per second
    """
    try:
        import imageio
        
        # Ensure frames are uint8
        frames_uint8 = [f.astype(np.uint8) for f in frames]
        
        imageio.mimsave(output_path, frames_uint8, fps=fps)
        print(f"Video saved to: {output_path}")
        return True
    except ImportError:
        print("imageio not installed. Install with: pip install imageio imageio-ffmpeg")
        return False
    except Exception as e:
        print(f"Error saving video: {e}")
        return False

# Save the rollout video
video_path = "/tmp/libero_rollout.mp4"
save_video(frames, video_path)

In [ ]:
# Display frames as animation (for Jupyter)
def display_rollout_frames(frames, step_size=5):
    """
    Display rollout frames in a grid.
    """
    # Sample frames
    sampled = frames[::step_size]
    n_frames = len(sampled)
    n_cols = min(5, n_frames)
    n_rows = (n_frames + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(3*n_cols, 3*n_rows))
    axes = np.array(axes).flatten()
    
    for i, (ax, frame) in enumerate(zip(axes, sampled)):
        ax.imshow(frame)
        ax.set_title(f"Step {i * step_size}")
        ax.axis('off')
    
    # Hide unused axes
    for ax in axes[len(sampled):]:
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

display_rollout_frames(frames, step_size=10)

---
## 10. Multiple Task Evaluation Setup

In [ ]:
def evaluate_on_suite(
    suite_name,
    policy_fn,
    n_trials_per_task=20,
    max_steps=400,
    verbose=True
):
    """
    Evaluate a policy on a full LIBERO task suite.
    
    Args:
        suite_name: Name of task suite (e.g., 'libero_spatial')
        policy_fn: Policy function(image, instruction) -> action
        n_trials_per_task: Number of trials per task
        max_steps: Maximum steps per episode
        verbose: Print progress
    
    Returns:
        results: Dict mapping task_id -> success_rate
    """
    task_names_list = get_task_names(suite_name)
    n_tasks = get_num_tasks(suite_name)
    results = {}
    
    for task_id in range(n_tasks):
        task_name = task_names_list[task_id]
        if verbose:
            print(f"\nEvaluating task {task_id}: {task_name}")
        
        # Get task and create environment
        task = get_task(suite_name, task_id)
        env = create_libero_env(task, suite_name)
        
        successes = 0
        for trial in range(n_trials_per_task):
            env.seed(trial)  # Different initial state
            
            success, _, _ = run_policy_rollout(
                env, policy_fn, task.language,
                max_steps=max_steps,
                record_video=False
            )
            
            successes += int(success)
            
            if verbose and trial % 5 == 0:
                print(f"  Trial {trial}/{n_trials_per_task}")
        
        success_rate = successes / n_trials_per_task
        results[task_id] = success_rate
        
        if verbose:
            print(f"  Success rate: {success_rate:.1%}")
        
        env.close()
    
    # Overall statistics
    mean_success = np.mean(list(results.values()))
    if verbose:
        print(f"\nOverall success rate: {mean_success:.1%}")
    
    return results

# Example: Quick evaluation on one task
print("Evaluation framework ready!")
print("Full evaluation will be demonstrated in notebook 08.")

---
## Summary

### LIBERO Setup Complete

1. **Environment**: MuJoCo-based simulation with Franka robot

2. **Task Suites**: 4 suites with 90 total tasks
   - libero_spatial (10 tasks)
   - libero_object (10 tasks)
   - libero_goal (10 tasks)
   - libero_90 (90 tasks)

3. **Observations**: 256×256 RGB agentview images

4. **Actions**: 7-DoF (position, rotation, gripper)

5. **Preprocessing**: Rotate 180°, JPEG encode/decode, resize to 224×224

### For Remote Server
- Set `MUJOCO_GL=osmesa` for CPU rendering
- Or `MUJOCO_GL=egl` for GPU-accelerated rendering
- Install: `sudo apt-get install libosmesa6-dev`

### Next Steps
→ Continue to **08_integrated_evaluation.ipynb** for full OpenVLA + LIBERO evaluation.

In [ ]:
# Clean up
env.close()
print("Environment closed.")